# Marketing Customer Segmentation Using RFM

In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


In [110]:
warnings.filterwarnings("ignore")

## Data Important & inspection

In [111]:
retail = pd.read_excel('Online Retail.xlsx')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [112]:
retail_df = retail.copy()
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [113]:
retail_df['Quantity'] = retail_df['Quantity'].astype('int16')
retail_df['UnitPrice'] = retail_df['UnitPrice'].astype('float32')
retail_df['CustomerID'] = retail_df['CustomerID'].astype('float32')
retail_df['StockCode'] = retail_df['StockCode'].astype('category')
retail_df['Country'] = retail_df['Country'].astype('category')

In [114]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  category      
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int16         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float32       
 6   CustomerID   406829 non-null  float32       
 7   Country      541909 non-null  category      
dtypes: category(2), datetime64[ns](1), float32(2), int16(1), object(2)
memory usage: 19.3+ MB


In [115]:
retail_df.describe().round(2)

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.00,541909,541909.00,406829.00
mean,9.55,2011-07-04 13:34:57.156386048,4.61,15287.69
min,-15459.00,2010-12-01 08:26:00,-11062.06,12346.00
25%,1.00,2011-03-28 11:34:00,1.25,13953.00
50%,3.00,2011-07-19 17:17:00,2.08,15152.00
75%,10.00,2011-10-19 11:27:00,4.13,16791.00
max,15459.00,2011-12-09 12:50:00,38970.00,18287.00
std,64.65,NaN,96.71,1713.57


## Data Preparation & Exploration

In [116]:
retail_df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [117]:
retail_df.dropna(inplace=True)

In [118]:
retail_df.shape

(406829, 8)

In [119]:
retail_df['Description']

0          WHITE HANGING HEART T-LIGHT HOLDER
1                         WHITE METAL LANTERN
2              CREAM CUPID HEARTS COAT HANGER
3         KNITTED UNION FLAG HOT WATER BOTTLE
4              RED WOOLLY HOTTIE WHITE HEART.
                         ...                 
541904            PACK OF 20 SPACEBOY NAPKINS
541905           CHILDREN'S APRON DOLLY GIRL 
541906          CHILDRENS CUTLERY DOLLY GIRL 
541907        CHILDRENS CUTLERY CIRCUS PARADE
541908          BAKING SET 9 PIECE RETROSPOT 
Name: Description, Length: 406829, dtype: object

In [120]:
retail_df.groupby('Description').agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head(10)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409
POPCORN HOLDER,30504
RABBIT NIGHT LIGHT,27094
MINI PAINT SET VINTAGE,25880
PACK OF 12 LONDON TISSUES,25321


In [121]:
retail_df['InvoiceNo'].str.contains('C').count()
#retail_df['InvoiceNo'].str.contains('C').sum()

8905

In [122]:
retail_df = retail_df[~retail_df['InvoiceNo'].str.contains('C', na=False)]

In [123]:
retail_df['TotalPrice'] = (retail_df['Quantity'] * retail_df['UnitPrice']).round(2)
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  category      
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int16         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float32       
 6   CustomerID   397924 non-null  float32       
 7   Country      397924 non-null  category      
 8   TotalPrice   397924 non-null  float32       
dtypes: category(2), datetime64[ns](1), float32(3), int16(1), object(2)
memory usage: 18.8+ MB


In [124]:
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## RFM Analysis

In [125]:
import datetime as dt


In [126]:
# Step 1: Get the latest invoice date in the dataset
latest_date = retail_df["InvoiceDate"].max()

In [127]:
# Step 2: Set today's date for recency calculation (as shown in your screenshot)
today_date = dt.datetime(2011, 12, 11)

In [128]:
# Step 3: Perform RFM aggregation
rfm = retail_df.groupby('CustomerID').agg({
    'InvoiceDate': lambda date: (today_date - date.max()).days,  # Recency
    'InvoiceNo': lambda num: num.nunique(),                      # Frequency
    'TotalPrice': lambda TotalPrice: TotalPrice.sum()            # Monetary
})
rfm

,InvoiceDate,InvoiceNo,TotalPrice
CustomerID,,,
12346.0,326,1,9026.160156
12347.0,3,7,4309.999512
12348.0,76,4,1797.239990
12349.0,19,1,1757.550049
12350.0,311,1,334.399994
...,...,...,...
18280.0,278,1,180.600006
18281.0,181,1,80.820007
18282.0,8,2,178.050003


In [129]:
# Step 4: Rename columns for clarity
rfm.columns = ["Recency", "Frequency", "Monetary"]

In [130]:
# Step 5: Filter out customers with zero monetary value (inactive or returns)
rfm = rfm[rfm["Monetary"] > 0]
rfm

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,1,9026.160156
12347.0,3,7,4309.999512
12348.0,76,4,1797.239990
12349.0,19,1,1757.550049
12350.0,311,1,334.399994
...,...,...,...
18280.0,278,1,180.600006
18281.0,181,1,80.820007
18282.0,8,2,178.050003


In [132]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4338.0,93.059474,100.012264,1.00,18.000000,51.000000,142.750000,374.00000
Frequency,4338.0,4.272706,7.706221,1.00,1.000000,2.000000,5.000000,210.00000
Monetary,4338.0,2007.131348,8563.699219,3.75,307.414993,674.484985,1661.739868,280206.03125


In [137]:
# 1. Recency Score: Lower recency = better → assign higher score (5 = most recent)
rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
# 2. Frequency Score: Higher frequency = better → assign higher score (5 = highest frequency)
# Use .rank(method="first") to handle ties by assigning unique ranks before qcut
rfm["frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
# 3. Monetary Score: Higher monetary value = better → assign higher score (5 = highest spend)
rfm["monetary_score"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5])

rfm.sample(10)

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score
CustomerID,,,,,,
14293.0,4,4,1046.559937,5,4,4
12913.0,5,5,2483.629883,5,4,5
14162.0,174,2,289.359985,2,3,2
13617.0,41,3,544.179993,3,3,3
12572.0,9,2,1020.419983,5,2,4
15651.0,37,3,646.300049,3,3,3
12421.0,16,4,807.039978,4,4,3
13754.0,152,2,282.600006,2,2,2
13481.0,46,5,1804.560059,3,4,4


In [141]:
# Create RFM_SCORE by combining recency_score and frequency_score as strings
rfm["RFM_SCORE"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
CustomerID,,,,,,,
12346.0,326,1,9026.160156,1,1,5,11
12347.0,3,7,4309.999512,5,5,5,55
12348.0,76,4,1797.239990,2,4,4,24
12349.0,19,1,1757.550049,4,1,4,41
12350.0,311,1,334.399994,1,1,2,11


In [145]:
# Define segmentation mapping using regex patterns
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_lose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

# Apply segmentation based on RFM_SCORE (string column)
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

# Display first few rows to verify
rfm.sample(5)

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
CustomerID,,,,,,,,
15147.0,241,1,623.499939,1,1,3,11,hibernating
17312.0,139,3,644.299988,2,4,3,24,at_risk
12673.0,6,2,610.520020,5,2,3,52,potential_loyalists
16224.0,58,4,1295.299927,3,4,4,34,loyal_customers
16976.0,268,1,251.519989,1,2,2,12,hibernating


In [146]:
# Group by segment and compute mean, count, max for Recency, Frequency, Monetary
segment_summary = (
    rfm[["segment", "Recency", "Frequency", "Monetary"]]
    .groupby("segment")
    .agg(["mean", "count", "max"])
    .round(1)  # Round to 1 decimal place for cleaner output
)

# Display the result
segment_summary

Recency            Frequency                Monetary  \
                       mean count  max      mean count  max         mean   
segment                                                                    
about_to_sleep         53.3   352   72       1.2   352    2   472.000000   
at_risk               153.8   593  374       2.9   593    6  1084.500000   
cant_lose             133.0    63  373       8.4    63   34  2796.199951   
champions               6.4   633   13      12.4   633  210  6857.899902   
hibernating           217.6  1071  374       1.1  1071    2   425.000000   
loyal_customers        33.6   819   72       6.5   819   63  2864.199951   
need_attention         52.4   187   72       2.3   187    3   897.599976   
new_customers           7.4    42   13       1.0    42    1   388.200012   
potential_loyalists    17.4   484   33       2.0   484    3   759.599976   
promising              23.5    94   33       1.0    94    1   294.000000   

                                          
                    count            max  
segment                                   
about_to_sleep        352    6207.700195  
at_risk               593   44534.300781  
cant_lose              63   10254.200195  
champions             633  280206.000000  
hibernating          1071    9864.299805  
loyal_customers       819  124914.500000  
need_attention        187   12601.799805  
new_customers          42    3861.000000  
potential_loyalists   484   32157.599609  
promising              94    1757.599976